<font size="+3"><strong>3.3. Autoregressive Models</strong></font>

# Import Necessary Packages

In [1]:
# import warnings

# import matplotlib.pyplot as plt
# import pandas as pd
# import plotly.express as px
# from IPython.display import VimeoVideo
# from pymongo import MongoClient
# from sklearn.metrics import mean_absolute_error
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.ar_model import AutoReg

# warnings.simplefilter(action="ignore", category=FutureWarning)

# Use Existing Wrangle Function

In [1]:
# def wrangle(collection):
#     results = collection.find(
#         {"metadata.site": 29, "metadata.measurement": "P2"},
#         projection={"P2": 1, "timestamp": 1, "_id": 0},
#     )

#     # Read data into DataFrame
#     df = pd.DataFrame(list(results)).set_index("timestamp")

#     # Localize timezone
#     df.index = df.index.tz_localize("UTC").tz_convert("Africa/Nairobi")

#     # Remove outliers
#     df = df[df["P2"] < 500]

#     # Resample to 1hr window
#     y = df["P2"].resample("1H").mean().fillna(method='ffill').to_frame()

#     return y["P2"]

In [2]:
# y = wrangle(nairobi)
# y.head()

# See Correlation of original data with LAG (ACF)

- Tells if current data can be predicted by any lag of the past
- Shows the correlation between current value and the values in the past

In [3]:
# print(y.corr(y))
# print(y.corr(y.shift(1)))
# print(y.corr(y.shift(2)))
# print(y.corr(y.shift(3)))

# See auto correlation plot

- Values on x-axis show lag values
- Values on y-axis show corresponding correlation of the lag related to current value 
- Correlation determines the correlation between current value and other lags
- Any values between the blue band area region is not statistically significant
- The lags with the most vertical height has the most significant predictive power 

In [4]:
# fig, ax = plt.subplots(figsize=(15, 6))
# plot_acf(y, ax=ax)
# plt.xlabel("Lag [hours]")
# plt.ylabel("Correlation Coefficient");

# See Correlation between different LAGs (echo)


In [5]:
# print(y.shift(1).corr(y.shift(2))) #echo of same correlation
# print(y.shift(2).corr(y.shift(3))) #echo of same correlation

# print(y.shift(1).corr(y.shift(3))) #echo of same correlation
# print(y.shift(2).corr(y.shift(4))) #echo of same correlation

# print(y.shift(1).corr(y.shift(4))) #echo of same correlation
# print(y.shift(2).corr(y.shift(5))) #echo of same correlation

# See partial auto correlation plot


- For every hour, the previous hour has some level of correlation
    - This correlation is same for 1PM and 2PM, 2PM and 3PM and so on..
    - This correlation is echoed in every dataset
    - Partial auto correlation determines the original correlation between values without this echo correlation
- Determines how many lags term will come in your model
- Values on x-axis show lag values
- Values on y-axis show corresponding correlation of the lag related to current value without echo
- Correlation determines the correlation between current value and other lags if the echo did not exist
- Any values between the blue band area region is not statistically significant
- The lags with the most vertical height has the most significant predictive power 


In [6]:
# fig, ax = plt.subplots(figsize=(15, 6))
# plot_pacf(y, ax=ax)
# plt.xlabel("Lag [hours]")
# plt.ylabel("Correlation Coefficient");

# Split Training and Testing Data

In [7]:
# int(len(y) * 0.95)

# cutoff_test = int(len(y) * 0.95)

# y_train = y.iloc[:cutoff_test]
# y_test = y.iloc[cutoff_test:]

# Create Baseline Model

In [13]:
# y_train_mean = y_train.mean()
# y_pred_baseline = [y_train_mean] * len(y_train)



# Mean Absolute Error of Baseline Model

In [14]:
# mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

# print("Mean P2 Reading:", round(y_train_mean, 2))
# print("Baseline MAE:", round(mae_baseline, 2))


# Create Auto-regression Model

In [9]:
# model = AutoReg(y_train, lags= 26).fit()

In [11]:
# model.predict().isnull().sum()

# Mean Absolute Error of Autoregressive Model

In [12]:
# y_pred = model.predict()
# training_mae = mean_absolute_error(y_train[26:], y_pred[26:])
# print("Training MAE:", training_mae)

# See residuals of the model

In [15]:
# y_train_resid = model.resid  # y_train - y_pred
# y_train_resid.tail()

# Plot Residuals

In [16]:
# fig, ax = plt.subplots(figsize=(15, 6))
# y_train_resid.plot(ylabel = "Residual Values", ax = ax)

# See distribution of residuals

In [17]:
# fig, ax = plt.subplots(figsize=(15, 6))
# y_train_resid.plot(kind = "hist", ax=ax)
# plt.xlabel("Residual Value")
# plt.ylabel("Frequency")
# plt.title("AR 26 Residual Distribution")

# See ACF Plot of residuals

In [18]:
# fig, ax = plt.subplots(figsize=(15, 6))
# plot_acf(y_train_resid, ax = ax)

# Find Mean Absolute Error of the model

In [19]:
# min_test = y_test.index.min()
# max_test = y_test.index.max()

In [20]:
# y_pred_test = model.predict(min_test, max_test)
# test_mae = mean_absolute_error(y_test, y_pred_test)
# print("Test MAE:", test_mae)

In [21]:
# df_pred_test = pd.DataFrame(
#     {"y_test": y_test, "y_pred": y_pred_test}, index=y_test.index
# )
# df_pred_test.head()

# Plot testing data vs predicted data

In [22]:
# fig = px.line(df_pred_test, labels={"value": "P2"})
# fig.show()

# Create a walk-forward Model

In [23]:
# %%capture

# y_pred_wfv = pd.Series()
# history = y_train.copy()
# for i in range(len(y_test)):
#     model = AutoReg(history, lags= 26).fit()
#     next_pred = model.forecast()
#     y_pred_wfv = y_pred_wfv.append(next_pred)
#     history = history.append(y_test[next_pred.index])

# Find MAE on walk-forward Model

In [24]:
# test_mae = mean_absolute_error(y_test, y_pred_wfv)
# print("Test MAE (walk forward validation):", round(test_mae, 2))

# See Model Parameters

In [25]:
# print(model.params)

# Plot test data vs Walk-Forward predicted data

In [26]:
# df_predictions = pd.DataFrame({
#     "y_test" : y_test,
#     "y_pred_wfv": y_pred_wfv
# })
# fig = px.line(df_predictions, labels = {"value" : "PM2.5"})
# fig.show()